# Курсовая работа № 2
## Аналитика. Начальный уровень

Курсовая работа состоит из двух частей – обязательной и дополнительной. **Для зачета необходимо выполнение только первой части.** Выполнение второй части может потребовать дополнительные знания Python.

- [Часть первая](#Часть-первая)
- [Часть вторая](#Часть-вторая)

## Часть первая


Перед вами стоит задача – подготовить аналитический отчет для HR-отдела. На основании проведенной аналитики предполагается составить рекомендации для отдела кадров по стратегии набора персонала, а также по взаимодействию с уже имеющимися сотрудниками.
<br><br> В базе данных лежит набор таблиц, которые содержат данные о сотрудниках вымышленной компании.
Сделайте обзор штата сотрудников компании. Составьте набор предметов исследования, а затем проверьте их на данных. Вся аналитика должна быть выполнена с помощью SQL. Впоследствии данные можно визуализировать, однако финальные датафреймы для графиков также должны быть подготовлены с помощью SQL. <br><br>

Примеры гипотез:
1. Есть зависимость между `performance score` и тем, под чьим руководством работает сотрудник.
2. Есть зависимость между продолжительностью работы в компании и семейным положением сотрудника.
2. Есть зависимость между продолжительностью работы в компании и возрастом сотрудника.

<br><br>
Параметры для подключения следующие: хост – `dsstudents.skillbox.ru`, порт – `5432`, имя базы данных – `human_resources`, пользователь – `readonly`, пароль – `6hajV34RTQfmxhS`. Таблицы, доступные для анализа, – `hr_dataset`, `production_staff`, `recruting_costs`, `salary_grid`.

In [1]:
import psycopg2
import pandas as pd

# подключаемся к базе данных, получаем объект типа connection

conn = psycopg2.connect(
    host='dsstudents.skillbox.ru', 
    user='readonly',
    password='6hajV34RTQfmxhS',
    port=5432,
    database='human_resources'
)

cursor = conn.cursor()  # объект cursor() для произведения запрососв в БД

Для того, чтобы наименования колонок и их типы по таблицам были перед глазами, выгрузим их заранее.

In [48]:
query_1 = "select column_name, data_type from information_schema.columns where table_name = 'production_staff';"
df1 = pd.read_sql_query(query_1, conn)
print(query_1.split("'")[1])
df1

production_staff


,column_name,data_type
0,id,integer
1,Employee Name,character varying
2,Race Desc,character varying
3,Date of Hire,date
4,TermDate,date
5,Reason for Term,character varying
6,Employment Status,character varying
7,Department,character varying
8,Position,character varying
9,Pay,money


In [49]:
query_2 = "select column_name, data_type from information_schema.columns where table_name = 'hr_dataset';"
df2 = pd.read_sql_query(query_2, conn)
print(query_2.split("'")[1])
df2

hr_dataset


,column_name,data_type
0,id,integer
1,Employee Name,character varying
2,Employee Number,integer
3,marriedid,integer
4,maritalstatusid,integer
5,genderid,integer
6,empstatus_id,integer
7,deptid,integer
8,perf_scoreid,integer
9,age,integer


In [50]:
query_3 = "select column_name, data_type from information_schema.columns where table_name = 'salary_grid';"
df3 = pd.read_sql_query(query_3, conn)
print(query_3.split("'")[1])
df3

salary_grid


,column_name,data_type
0,id,integer
1,Position,character varying
2,Salary Min,integer
3,Salary Mid,integer
4,Salary Max,integer
5,Hourly Min,real
6,Hourly Mid,real
7,Hourly Max,real


Проверим следующую гипотезу:<br>
### H1. Зарплата сотрудников прямым образом зависит от оценки производительности (*Pa* и *Performance Score*)

По какому принципу:<br>
существует определенная градация оценок производительности, которые по престижности представляют из себя следующий список (толщина букв чередуется для удобства чтения):<br>
1. **Exceptional**
2. Exceeds
3. **Fully meets**
4. 90-day meets
5. **Needs Improvement**
6. N/A - too early to review

Узнаем, действительно ли зарплаты сотрудников распределены согласно этой шкале. Для этого сравним списки зарплат и оценок производительнсти на первых 50 примерах.

In [51]:
# cursor.execute("SELECT datname FROM pg_database;") # список БД
# cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'") # список таблиц
# cursor.execute("select column_name, data_type from information_schema.columns where table_name = 'production_staff';") 
# список наименований столбцов таблицы production_staff

import psycopg2
import pandas as pd


query = """SELECT "Performance Score", "Pay"
           FROM production_staff
           WHERE "Performance Score" != ''
           ORDER BY "Pay" DESC
           LIMIT 50;"""

query_mean = """SELECT "Performance Score", MAX("Pay") as "Max", MIN("Pay") as "Min"
                FROM production_staff
                WHERE "Performance Score" != ''
                GROUP BY "Performance Score";"""

with psycopg2.connect(host='dsstudents.skillbox.ru',       # для того, чтобы каждый раз не отключать соединение 
                      user='readonly',                     # с БД и не подключаться к ней вручную,
                      password='6hajV34RTQfmxhS',          # будем работать в контекстном менеджере
                      port=5432,                           
                      database='human_resources') as conn:
    with conn.cursor() as cursor:                          # так же и для объекта cursor()
        cursor.execute(query)
        result = cursor.fetchall()
        print([_ for _ in result])
        df_pays = pd.DataFrame({"Performance Score":[_[0] for _ in result], "Pay": [_[1] for _ in result]})
    
        print()
        cursor.execute(query_mean)
        result = cursor.fetchall()
        print([_ for _ in result])
        df_mean_pays = pd.DataFrame({"Performance Score":[_[0] for _ in result],\
                                     "Max pay": [_[1] for _ in result],\
                                     "Min pay": [_[2] for _ in result]})  

[('Exceeds', '$55.00'), ('Exceeds', '$55.00'), ('Fully Meets', '$55.00'), ('Fully Meets', '$54.50'), ('Fully Meets', '$54.00'), ('Needs Improvement', '$53.00'), ('Fully Meets', '$53.00'), ('Fully Meets', '$52.00'), ('Fully Meets', '$51.00'), ('Fully Meets', '$50.50'), ('Fully Meets', '$48.00'), ('Needs Improvement', '$42.00'), ('Fully Meets', '$38.00'), ('Fully Meets', '$33.50'), ('PIP', '$29.00'), ('Fully Meets', '$29.00'), ('Fully Meets', '$29.00'), ('Exceeds', '$29.00'), ('Fully Meets', '$29.00'), ('Exceeds', '$29.00'), ('Fully Meets', '$29.00'), ('90-day meets', '$29.00'), ('Fully Meets', '$28.75'), ('N/A- too early to review', '$28.00'), ('N/A- too early to review', '$28.00'), ('Fully Meets', '$28.00'), ('Fully Meets', '$28.00'), ('Fully Meets', '$27.00'), ('Fully Meets', '$27.00'), ('Fully Meets', '$27.00'), ('Needs Improvement', '$27.00'), ('Fully Meets', '$27.00'), ('Exceeds', '$27.00'), ('Fully Meets', '$26.39'), ('Exceeds', '$26.10'), ('N/A- too early to review', '$26.00'), (

In [52]:
# Для читабельности сформируем из получившихся значений датафреймы
df_pays

,Performance Score,Pay
0,Exceeds,$55.00
1,Exceeds,$55.00
2,Fully Meets,$55.00
3,Fully Meets,$54.50
4,Fully Meets,$54.00
5,Needs Improvement,$53.00
6,Fully Meets,$53.00
7,Fully Meets,$52.00
8,Fully Meets,$51.00
9,Fully Meets,$50.50


In [53]:
df_mean_pays

,Performance Score,Max pay,Min pay
0,Exceptional,$25.00,$17.00
1,90-day meets,$29.00,$15.00
2,N/A- too early to review,$28.00,$15.00
3,Fully Meets,$55.00,$14.00
4,Exceeds,$55.00,$15.00
5,PIP,$29.00,$16.00
6,Needs Improvement,$53.00,$15.00


Видим, что распределение зарплат не соответствует указанному выше "табелю о рангах". Возможно, из можно объяснить 
должностями, сложности которых в корне отличны дркг от друга. Например, есть работы настолько сложные, что статус
"требует улучшения" в них ценится выше, чем статус "исключительный" в более простых видах деятельности.

Решение: **гипотеза H1 отвергается**.

Проверим следующую гипотезу:<br>
### H2. Возраст сотрудников зависит от продолжительности работы в компании (*age* и *Days Employed*)

In [54]:
#     hr_dataset, production_staff, recruting_costs, salary_grid
query = """SELECT "age",  AVG("Days Employed")
           FROM hr_dataset
           WHERE "Employee Name"!=''
           GROUP BY "age"
           ORDER BY "age";"""

with psycopg2.connect(host='dsstudents.skillbox.ru',       
                      user='readonly',                     
                      password='6hajV34RTQfmxhS',          
                      port=5432,                           
                      database='human_resources') as conn:
    with conn.cursor() as cursor:                          
        cursor.execute(query)
        result = cursor.fetchall()
        print([_ for _ in result])
        df_age_and_dur = pd.DataFrame({"Age":[_[0] for _ in result], "Working duration": [round(_[1], 2) for _ in result]})

[(25, Decimal('1843.5000000000000000')), (26, Decimal('849.5000000000000000')), (27, Decimal('1547.0000000000000000')), (28, Decimal('962.5833333333333333')), (29, Decimal('1626.4666666666666667')), (30, Decimal('1093.5333333333333333')), (31, Decimal('1200.0800000000000000')), (32, Decimal('1309.4166666666666667')), (33, Decimal('1059.2352941176470588')), (34, Decimal('1098.1052631578947368')), (35, Decimal('1540.3846153846153846')), (36, Decimal('1237.4615384615384615')), (37, Decimal('1155.1250000000000000')), (38, Decimal('1562.6666666666666667')), (39, Decimal('1440.0000000000000000')), (40, Decimal('1709.1250000000000000')), (41, Decimal('1278.5000000000000000')), (42, Decimal('1470.6250000000000000')), (43, Decimal('1251.2222222222222222')), (44, Decimal('1221.8888888888888889')), (45, Decimal('1152.7500000000000000')), (46, Decimal('299.3333333333333333')), (47, Decimal('1384.5000000000000000')), (48, Decimal('1269.6000000000000000')), (49, Decimal('1995.0000000000000000')), (5

In [55]:
df_age_and_dur

,Age,Working duration
0,25,1843.50
1,26,849.50
2,27,1547.00
3,28,962.58
4,29,1626.47
5,30,1093.53
6,31,1200.08
7,32,1309.42
8,33,1059.24
9,34,1098.11


Видим, что никакой зависимости между возрастом сотрудника и тем, как долго он работает в компании нет.<br><br>
Решение: **гипотеза H2 отвергается**.

Проверим следующую гипотезу:<br>
### H3. Зарплаты инженеров меньше, чем зарплаты руководителей (*Position* и *Pay Rate*)

In [56]:
query = """SELECT "position",  AVG("Pay Rate") as "AVG_SALARY"
           FROM hr_dataset
           WHERE "Employee Name"!=''
           GROUP BY "position"
           ORDER BY "AVG_SALARY";"""

with psycopg2.connect(host='dsstudents.skillbox.ru',       
                      user='readonly',                     
                      password='6hajV34RTQfmxhS',          
                      port=5432,                           
                      database='human_resources') as conn:
    with conn.cursor() as cursor:                          
        cursor.execute(query)
        result = cursor.fetchall()
        print([_ for _ in result])
        df_age_and_dur = pd.DataFrame({"Age":[_[0] for _ in result], "Working duration": [round(_[1], 2) for _ in result]})

[('Production Technician I', 19.1044852943981), ('Administrative Assistant', 19.5199998219808), ('Production Technician II', 25.4077192942301), ('Accountant I', 26.8333333333333), ('Software Engineering Manager', 27.0), ('IT Support', 28.4699997901917), ('Sr. Accountant', 34.9500007629395), ('Database Administrator', 39.4769229888916), ('Network Engineer', 39.6777776082357), ('IT Manager - DB', 41.5), ('BI Developer', 45.25), ('Production Manager', 49.6785714285714), ('Software Engineer', 51.0722215440538), ('Senior BI Developer', 51.1666666666667), ('Sr. Network Engineer', 54.4599998474121), ('Shared Services Manager', 55.0), ('Data Architect', 55.0), ('Area Sales Manager', 55.2222222222222), ('Sales Manager', 56.75), ('Director of Operations', 60.0), ('Director of Sales', 60.0), ('Sr. DBA', 60.0), ('IT Manager - Infra', 63.0), ('BI Director', 63.5), ('IT Manager - Support', 64.0), ('IT Director', 65.0), ('CIO', 65.0), ('President & CEO', 80.0)]


Полученные данные убеждают нас в том, что менеджеры получают больше технарей.<br><br>
Решение: **гипотеза H3 принимается**.

## Часть вторая

Перед вами стоит задача – подготовить аналитический ответ для SMM-отдела компании Skillbox. <br> Объектом анализа является  [паблик Skillbox Вконтакте](https://vk.com/skillbox_education). <br> <br> 
Подключитесь к  API VK и выгрузите посты со стены паблика Skillbox за интересующий период (определите самостоятельно и обоснуйте). Проанализируйте влияние различных факторов (например, времени публикации) на вовлеченность пользователей (количество лайков, комментариев, голосов в опросах). Сделайте аналитику по рубрикам (примеры рубрик: дизайн-битва, игра по управлению), которые есть в паблике. Выбрать нужные посты можно с помощью регулярных выражений. Составьте перечень рекомандаций для SMM-отдела по итогам анализа. <br> <br> 

Дополнительные инструкции по работе с API VK расположены [здесь](https://nbviewer.jupyter.org/github/vikaborel/dstutorials/blob/master/Дополнительные%20инструкции.ipynb).

In [59]:
import requests

def getjson(url):
    response = requests.get(url)
    response = response.json()
    return response

In [297]:
vk_token = '6e7445a0b9c7ab289f1ee8de6d8b3850cff86965da9d61b7b45163a6c059855c40c699539c1dafa70fb34'
url = 'https://api.vk.com/method/account.getProfileInfo?access_token=' + str(vk_token) + '&v=5.92'
response = getjson(url)
response

{'response': {'first_name': 'Жираслан',
  'id': 443612597,
  'last_name': 'Шеджем',
  'home_town': 'Нальчик',
  'status': '',
  'bdate': '7.9.1999',
  'bdate_visibility': 0,
  'phone': '+7 *** *** ** 79',
  'relation': 0,
  'screen_name': 'zhiraslan_shedzhem',
  'sex': 2}}

Решено было провести аналитический обзор за период, продолжавшийся в течение последнего года до возникновения так напугавшей весь мир пандемии коронавируса, а именно - до дня объявления карантина по всей России (25.03.2020). Этот 
год был выбран из-за того, что это самый недавний отрезок времени, в котором предприятия по всей стране работали 
(и студенты учились) в обычном для себя режиме.

In [298]:
quarantione_day = datetime.strptime('2020-03-25', '%Y-%m-%d')   # опорная дата в нужном формате
period = 0
dates = []   # массив дат

print(f'Сколько постов добавлено:')
while period < 1000:
    url = f'https://api.vk.com/method/wall.get?access_token=' + str(vk_token) +\
    '&owner_id=-66669811&offset={}&count=100&v=5.92'.format(period)
    response = getjson(url)
    
    for _ in range(len(list(response.values())[0]['items'])):
        dates.append(list(list(response.values())[0]['items'][_].items())[3][1])
    period += 100
    print(period)

print(f'\nВ нашу базу отобрано {len(dates)} постов.')

Сколько постов добавлено:
100
200
300
400
500
600
700
800
900
1000

В нашу базу отобрано 1000 постов.


In [299]:
from datetime import datetime, timedelta

for _ in range(len(dates)):   # переводим все даты в читаемый формат
    dates[_] = (datetime.fromtimestamp(dates[_]) - timedelta(hours=0)).strftime('%Y-%m-%d')

date_difference = (quarantione_day-datetime.strptime(dates[-1], '%Y-%m-%d')).days

correct = 'дней' if date_difference % 10 > 4 else 'дня'  # для корректного вывода сообщения

print(f'Дата публикации последнего отобранного поста\
 была за {date_difference} {correct} до объявления карантина.')

Дата публикации последнего отобранного поста была за 364 дня до объявления карантина.


In [300]:
import re

period = 0
likes = []  # списка с количествами лайков у записей
comments = []
reposts = [] 
views = []  # --||-- (просмотры)
tags = []  # --||-- (хештеги)

while period < 1000:
    url = f'https://api.vk.com/method/wall.get?access_token=' + str(vk_token) +\
    '&owner_id=-66669811&offset={}&count=100&v=5.92'.format(period)
    response = getjson(url)

    for _ in range(len(list(response.values())[0]['items'])):
        likes.append(list(response.values())[0]['items'][_]['likes']['count'])
        comments.append(list(response.values())[0]['items'][_]['comments']['count'])
        reposts.append(list(response.values())[0]['items'][_]['reposts']['count'])
        views.append(list(response.values())[0]['items'][_]['views']['count'])
        
        
        text = list(response.values())[0]['items'][_]['text']   # текст поста, в конце которого расположен хештег
        if len(re.findall(reg_ex, text))==0:
            tags.append(' ')
        else:
            tags.append(re.findall(reg_ex, text)[0].replace('_',"").replace(' ',"").lower())   # избавляемся от пробелов, 
            # подчеркиваний и приводим все символы к нижнему регистру
        
    period += 100
    print(period)

100
200
300
400
500
600
700
800
900
1000


In [70]:
import csv

f_name = "vk_api.csv"

columns = [['Дата',
            'Хештег',
            'Кол-во лайков',
            'Кол-во комментариев',
            'Кол-во репостов',
            'Кол-во просмотров']]

# запись в .csv-файл

with open(f_name, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file, delimiter = ",")
    writer.writerows(columns)
    for _ in range(len(dates)):
        writer.writerows([[dates[_], tags[_], likes[_], comments[_], reposts[_],  views[_]]])

In [301]:
import pandas as pd

# создание в датафрейм и фильтрация значений

df_vk = pd.read_csv("vk_api.csv")
df_vk.sort_values(by='Хештег', inplace=True)
df_vk = df_vk[df_vk['Хештег']!=' ']
df_vk

,Дата,Хештег,Кол-во лайков,Кол-во комментариев,Кол-во репостов,Кол-во просмотров
164,2020-04-01,aicskillbox,22,14,2,14303
998,2019-03-28,answer,4,43,0,9401
846,2019-05-16,answer,11,36,1,9886
768,2019-06-13,answer,11,64,0,9161
788,2019-06-06,answer,15,47,0,10018
...,...,...,...,...,...,...
19,2020-07-01,шрифтыskillbox,111,5,11,22247
409,2019-11-24,шрифтыskillbox,48,0,6,11060
150,2020-04-10,шрифтыskillbox,45,51,2,21147
87,2020-05-24,шрифтыskillbox,86,4,9,21995


In [306]:
# статистические подсчеты данных по основным направлениям

df_fields = df_vk[df_vk['Хештег'].str.contains('skillboxмаркетинг') | df_vk['Хештег'].str.contains('skillboxуправление') |\
         df_vk['Хештег'].str.contains('skillboxдизайн') | df_vk['Хештег'].str.contains('skillboxпрограммирование')]

df_fields = df_fields .groupby('Хештег').agg({'Кол-во лайков':'mean',\
                             'Кол-во комментариев':'mean',\
                             'Кол-во репостов':'mean',\
                             'Кол-во просмотров':'mean'})

df_fields.index = ['Дизайн', 'Маркетинг', 'Программирование', 'Управление']
df_fields = df_fields.apply('round')

# отправка DF в .csv-файл
df_fields.to_csv('fields.csv', sep=',')

df_fields

,Кол-во лайков,Кол-во комментариев,Кол-во репостов,Кол-во просмотров
Дизайн,116.0,9.0,13.0,19083.0
Маркетинг,72.0,11.0,9.0,16062.0
Программирование,102.0,13.0,10.0,20095.0
Управление,168.0,12.0,30.0,26824.0


In [319]:
# самые популярные хештеги

df_vk = df_vk.groupby('Хештег').agg({'Кол-во лайков':'mean',\
                             'Кол-во комментариев':'mean',\
                             'Кол-во репостов':'mean',\
                             'Кол-во просмотров':'mean'})

best_by_likes = df_vk.sort_values(by='Кол-во лайков')[:-6:-1].index
best_by_comments = df_vk.sort_values(by='Кол-во комментариев')[:-6:-1].index
best_by_reposts = df_vk.sort_values(by='Кол-во репостов')[:-6:-1].index
best_by_views = df_vk.sort_values(by='Кол-во просмотров')[:-6:-1].index

In [320]:
df_avg
bests = [list(best_by_likes), list(best_by_comments), list(best_by_reposts), list(best_by_views)]
df_avg = pd.DataFrame(zip(list(df_vk.columns), bests),columns = ['category', 'tags'])
df_avg

,category,tags
0,Кол-во лайков,"[айдентикskillbox, collections, учимhtml, урок..."
1,Кол-во комментариев,"[качаемкреатив, конкурсskillbox, skillboxafp, ..."
2,Кол-во репостов,"[collections, skillbox, cоветпреподаskillbox, ..."
3,Кол-во просмотров,"[skillboxsreda, интенсивskillbox, дизайнупаков..."


In [321]:
# запись в .csv-файл
f_name = 'average_bests.csv'

with open(f_name, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file, delimiter = ",")
    writer.writerows([['category', '1st', '2nd', '3rd', '4th', '5th']])
    for _ in range(len(df_avg)):
        writer.writerows([[df_avg.category[_], *df_avg.tags[_]]])

Согласно результатам исследования:

1. Самым популярным для подписчиков паблика направлением является **Дизайн** - 1 место по числу лайков (168), репостов (30) и просмотров (почти 3 тыс.); второе по обсуждаемости в комментариях (12).
2. Больше всего дискуссий вызывает хештег о **Программировании** - 13 комментов в среднем.
3. Наименее интересные посты о направлении **Маркетинг** - единственное, у которого нет и 100 лайков в среднем.
4. Фаворит по лайкам независимо от направления - *#Айдентик_skillbox*, 393 лайка.
5. По комментариям - *#Качаем_креатив*, 125 комментов.
6. По репостам - *#Collections*, 73 репоста в среднем.
8. По просмотрам - *#Skillbox_sreda*, 65368 просмотров.